In [1]:
import os
import sys
import math
import random

import numpy as np
import pandas as pd
import scipy.io as sio
from scipy.io import loadmat

from sklearn import preprocessing
from scipy.integrate import simps
from scipy.signal import butter, lfilter, welch

from tensorflow.keras.utils import to_categorical

In [2]:
# 巴特沃斯滤波器
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data) 
    return y

In [3]:
# 读取文件
def read_file(file):
    data = sio.loadmat(file)
    data = data['data']
    data_1st = loadmat(file)['data'][0]

    # print("读取data_preprocessed_matlab数据,打印第一个数据")
    # print(data.shape)
    # print("First data point:", data_1st)
    # print("")
    return data

In [ ]:
def compute_PSD(window_signal, low, high, fs=128):
    # 1. 计算功率谱密度 (PSD)
    # 对于0.5秒窗口(64个点)，使用适当的nperseg值
    # nperseg=64会给出单一窗口估计，nperseg=32会有更好的频率分辨率但更多方差
    freqs, psd = welch(window_signal, fs=fs, nperseg=64, 
                             scaling='density', average='mean')
    
    # 根据波段范围 (14-31Hz) 确定PSD中对应的频率索引
    # 虽然信号已经过滤波，但我们仍需要确定PSD中对应的频率索引
    idx_band = np.logical_and(freqs >= low, freqs <= high)
    
    # 3. 计算频率分辨率，即功率谱密度(PSD)中相邻频率点之间的间隔。
    freq_res = freqs[1] - freqs[0]
    
    # 4. 使用Simpson积分法计算β波段功率
    # 如果滤波非常精确，这基本上是整个PSD的积分
    beta_power = simps(psd[idx_band], dx=freq_res)
    
    return beta_power  # 返回单个数值


In [5]:
def decompose(file):
    print("decompose:") # 函数开始运行
    # trial*channel*sample
    # start_index 用于确定从信号中提取的时间范围的起始点。具体来说，它是用于指定从每个信号中提取的信号部分的开始位置。
    # 这里 start_index 的值为 384，表示在每个试验信号中，从第 384 个样本点开始提取信号。
    # 这3秒是预实验基线
    start_index = 384  # 3s pre-trial signals 128(hz)x3(s)=384 
    data = read_file(file)
    # 确定信号的形状 shape 和采样频率 frequency
    shape = data.shape
    frequency = 128

    decomposed_psd = np.empty([0, 4, 120])
    
    base_PSD = np.empty([0, 128])
    # 这里的range(40) 是因为有40个试验信号
    for trial in range(40):
        temp_base_PSD = np.empty([0])
        temp_base_theta_PSD = np.empty([0])
        temp_base_alpha_PSD = np.empty([0])
        temp_base_beta_PSD = np.empty([0])
        temp_base_gamma_PSD = np.empty([0])

        temp_psd = np.empty([0, 120])

        # 对于每个试验信号的每个通道，获取试验信号的部分（从第 384 个样本点开始，总共 60x128=7680 个）和基线信号的部分（前 384 个样本点）
        # 这里的range(32) 是因为前32个channel是EEG数据
        for channel in range(32):
            trial_signal = data[trial, channel, start_index:]
            base_signal = data[trial, channel, :start_index]
            # ****************compute base PSD****************
            # 使用了巴特沃斯滤波器对基线信号进行频带滤波。
            # 具体来说，对基线信号 base_signal 分别进行了四个频带的滤波，
            # 分别是 theta (4-8 Hz)、alpha (8-14 Hz)、beta (14-31 Hz) 和 gamma (31-45 Hz)。
            base_theta = butter_bandpass_filter(base_signal, 4, 8, frequency, order=3)
            base_alpha = butter_bandpass_filter(base_signal, 8, 14, frequency, order=3)
            base_beta = butter_bandpass_filter(base_signal, 14, 31, frequency, order=3)
            base_gamma = butter_bandpass_filter(base_signal, 31, 45, frequency, order=3)

            base_theta_PSD = (compute_PSD(base_theta[:64], 4, 8) + compute_PSD(base_theta[64:128], 4, 8) + compute_PSD(
                base_theta[128:192], 4, 8) + compute_PSD(base_theta[192:256], 4, 8) + compute_PSD(base_theta[256:320], 4, 8) + compute_PSD(
                base_theta[320:], 4, 8)) / 6
            base_alpha_PSD = (compute_PSD(base_alpha[:64], 8, 14) + compute_PSD(base_alpha[64:128], 8, 14) + compute_PSD(
                base_alpha[128:192], 8, 14) + compute_PSD(base_theta[192:256], 8, 14) + compute_PSD(base_theta[256:320], 8, 14) + compute_PSD(
                base_theta[320:], 8, 14)) / 6
            base_beta_PSD = (compute_PSD(base_beta[:64], 14, 31) + compute_PSD(base_beta[64:128], 14, 31) + compute_PSD(
                base_beta[128:192], 14, 31) + compute_PSD(base_theta[192:256], 14, 31) + compute_PSD(base_theta[256:320], 14, 31) + compute_PSD(
                base_theta[320:], 14, 31)) / 6
            base_gamma_PSD = (compute_PSD(base_gamma[:64], 31, 45) + compute_PSD(base_gamma[64:128], 31, 45) + compute_PSD(
                base_gamma[128:192], 31, 45) + compute_PSD(base_theta[192:256], 31, 45) + compute_PSD(base_theta[256:320], 31, 45) + compute_PSD(
                base_theta[320:], 31, 45)) / 6

            # 将基线信号在不同频带的微分熵数据添加到相应的临时数组中
            temp_base_theta_PSD = np.append(temp_base_theta_PSD, base_theta_PSD)
            temp_base_gamma_PSD = np.append(temp_base_gamma_PSD, base_alpha_PSD)
            temp_base_beta_PSD = np.append(temp_base_beta_PSD, base_beta_PSD)
            temp_base_alpha_PSD = np.append(temp_base_alpha_PSD, base_gamma_PSD)

            # 对试验信号进行巴特沃斯带通滤波，原理和基线信号一样
            theta = butter_bandpass_filter(trial_signal, 4, 8, frequency, order=3)
            alpha = butter_bandpass_filter(trial_signal, 8, 14, frequency, order=3)
            beta = butter_bandpass_filter(trial_signal, 14, 31, frequency, order=3)
            gamma = butter_bandpass_filter(trial_signal, 31, 45, frequency, order=3)

            # 将这四个数组初始化为全零数组，用于存储试验信号在不同频带下的功率谱密度数据
            PSD_theta = np.zeros(shape=[0], dtype=float)
            PSD_alpha = np.zeros(shape=[0], dtype=float)
            PSD_beta = np.zeros(shape=[0], dtype=float)
            PSD_gamma = np.zeros(shape=[0], dtype=float)

            # 这里实际上是把试验信号分成120个相对短的时间段或窗口，然后对每个窗口进行功率谱密度的计算。
            # 每个窗口的长度为64个样本点，窗口持续时间为0.5秒
            for index in range(120):
                PSD_theta = np.append(PSD_theta, compute_PSD(theta[index * 64:(index + 1) * 64], 4, 8))
                PSD_alpha = np.append(PSD_alpha, compute_PSD(alpha[index * 64:(index + 1) * 64], 8, 14))
                PSD_beta = np.append(PSD_beta, compute_PSD(beta[index * 64:(index + 1) * 64], 14, 31))
                PSD_gamma = np.append(PSD_gamma, compute_PSD(gamma[index * 64:(index + 1) * 64], 31, 45))

            # 通过 np.vstack 函数将每个频带下的功率谱密度数据垂直堆叠在一起，
            # 形成一个临时的二维数组 temp_psd。这个数组的每一行代表一个频带下的功率谱密度数据。
            temp_psd = np.vstack([temp_psd, PSD_theta])
            temp_psd = np.vstack([temp_psd, PSD_alpha])
            temp_psd = np.vstack([temp_psd, PSD_beta])
            temp_psd = np.vstack([temp_psd, PSD_gamma])

        temp_trial_psd = temp_psd.reshape(-1, 4, 120)
        decomposed_psd = np.vstack([decomposed_psd, temp_trial_psd])

        temp_base_PSD = np.append(temp_base_theta_PSD, temp_base_alpha_PSD)
        temp_base_PSD = np.append(temp_base_PSD, temp_base_beta_PSD)
        temp_base_PSD = np.append(temp_base_PSD, temp_base_gamma_PSD)

        # base_PSD 将包含所有通道在四个频带下的基线功率谱数据。
        base_PSD = np.vstack([base_PSD, temp_base_PSD])
        print("base_PSD:", base_PSD.shape)

    #40 视频 x 60秒/视频 / 0.5s (窗口) = 4800
    # 在这里要把每个视频的数据分开来计算
    decomposed_psd = decomposed_psd.reshape(-1, 32, 4, 120).transpose([0, 3, 2, 1]).reshape(40, 120, -1) # (40, 120, 128)
    
    print("base_PSD shape:", base_PSD.shape)
    print("trial_PSD shape:", decomposed_psd.shape)
    print("")
    return base_PSD, decomposed_psd

In [6]:
def get_labels(file):
    # 0 valence, 1 arousal, 2 dominance, 3 liking
    valence_labels = sio.loadmat(file)["labels"][:, 0] > 5  # valence labels
    arousal_labels = sio.loadmat(file)["labels"][:, 1] > 5  # arousal labels
    valence_labels = valence_labels.astype(int)
    arousal_labels = arousal_labels.astype(int)
    
    final_valence_labels = np.empty((40, 120))
    final_arousal_labels = np.empty((40, 120))
    
    for i in range(0, 40):
        final_valence_labels[i, :] = valence_labels[i]
        final_arousal_labels[i, :] = arousal_labels[i]
    print("get_labels:")
    print("labels:", final_arousal_labels.shape)
    return final_arousal_labels, final_valence_labels

# 输出已经提取出PSD特征的一维数据

In [7]:
# 输出psd文件
dataset_dir = "D:/huangzhiying/cross-video-emotion-recognition/cross-video-emotion-recognition/dataset/DEAP/raw_data"

result_dir = "D:/huangzhiying/cross-video-emotion-recognition/cross-video-emotion-recognition/dataset/DEAP/deap_1d_result/psd/"
if os.path.isdir(result_dir) == False:
    os.makedirs(result_dir)

for file in os.listdir(dataset_dir):
    print("processing: ", file, "......")
    file_path = os.path.join(dataset_dir, file)
    base_DE, trial_DE = decompose(file_path)
    arousal_labels, valence_labels = get_labels(file_path)
    sio.savemat(result_dir + "PSD_" + file,
                {"base_data": base_DE, "data": trial_DE, "valence_labels": valence_labels,
                    "arousal_labels": arousal_labels})

processing:  s01.mat ......
decompose:


C:\Users\ma-lab_2\AppData\Local\Temp\ipykernel_8116\403524263.py:27: DeprecationWarning: 'scipy.integrate.simps' is deprecated in favour of 'scipy.integrate.simpson' and will be removed in SciPy 1.14.0
  beta_power = simps(psd[idx_band], dx=freq_res)


base_PSD: (1, 128)
base_PSD: (2, 128)
base_PSD: (3, 128)
base_PSD: (4, 128)
base_PSD: (5, 128)
base_PSD: (6, 128)
base_PSD: (7, 128)
base_PSD: (8, 128)
base_PSD: (9, 128)
base_PSD: (10, 128)
base_PSD: (11, 128)
base_PSD: (12, 128)
base_PSD: (13, 128)
base_PSD: (14, 128)
base_PSD: (15, 128)
base_PSD: (16, 128)
base_PSD: (17, 128)
base_PSD: (18, 128)
base_PSD: (19, 128)
base_PSD: (20, 128)
base_PSD: (21, 128)
base_PSD: (22, 128)
base_PSD: (23, 128)
base_PSD: (24, 128)
base_PSD: (25, 128)
base_PSD: (26, 128)
base_PSD: (27, 128)
base_PSD: (28, 128)
base_PSD: (29, 128)
base_PSD: (30, 128)
base_PSD: (31, 128)
base_PSD: (32, 128)
base_PSD: (33, 128)
base_PSD: (34, 128)
base_PSD: (35, 128)
base_PSD: (36, 128)
base_PSD: (37, 128)
base_PSD: (38, 128)
base_PSD: (39, 128)
base_PSD: (40, 128)
base_PSD shape: (40, 128)
trial_PSD shape: (40, 120, 128)

get_labels:
labels: (40, 120)
processing:  s02.mat ......
decompose:
base_PSD: (1, 128)
base_PSD: (2, 128)
base_PSD: (3, 128)
base_PSD: (4, 128)
base_

# Get 2D data

In [8]:
# data(40*120*128)
# base_data(40*128)
# arousal_labels(40*120)
# valence_labels(40*120)
def read_file(file):
    file = sio.loadmat(file)
    trial_data = file['data']
    base_data = file["base_data"]
    return trial_data, base_data, file["arousal_labels"], file["valence_labels"]

##### 计算试验数据和基准数据之间的偏移

In [9]:
def get_vector_deviation(vector1, vector2):
    return vector1 - vector2

def get_dataset_deviation(trial_data, base_data):
    new_dataset = np.empty([0, 120, 128])
    for i in range(0, 40):
        new_record = np.array([get_vector_deviation(trial_data[i][j], base_data[i]) for j in range(0, 120)])
        # print("new_record shape:", new_record.shape)
        new_record = new_record[np.newaxis, :, :]  # 添加一个额外的维度
        new_dataset = np.vstack([new_dataset, new_record])
    # print("get_dataset_deviation:")
    # print("new_dataset shape:", new_dataset.shape) # new_dataset shape: (40, 120, 128)
    # print("new_dataset:" ,new_dataset)
    return new_dataset

In [10]:
def data_1Dto2D(data, Y=8, X=9):

    # print("data_1Dto2D data shape:", data.shape)

    data_2D = np.zeros([Y, X])
    data_2D[0] = (0, 0, data[1], data[0], 0, data[16], data[17], 0, 0)
    data_2D[1] = (data[3], 0, data[2], 0, data[18], 0, data[19], 0, data[20])
    data_2D[2] = (0, data[4], 0, data[5], 0, data[22], 0, data[21], 0)
    data_2D[3] = (data[7], 0, data[6], 0, data[23], 0, data[24], 0, data[25])
    data_2D[4] = (0, data[8], 0, data[9], 0, data[27], 0, data[26], 0)
    data_2D[5] = (data[11], 0, data[10], 0, data[15], 0, data[28], 0, data[29])
    data_2D[6] = (0, 0, 0, data[12], 0, data[30], 0, 0, 0)
    data_2D[7] = (0, 0, 0, data[13], data[14], data[31], 0, 0, 0)
    # return shape:9*9
    return data_2D

In [11]:
def pre_process(path, y_n):
    # DE feature vector dimension of each band
    data_3D = np.empty([0, 120, 4, 8, 9])
    sub_vector_len = 32
    trial_data, base_data, arousal_labels, valence_labels = read_file(path)
    if y_n == "yes":
        data = get_dataset_deviation(trial_data, base_data)

        # 将三维数组转换为二维数组
        reshaped_data = data.reshape(-1, data.shape[-1])
        # 对二维数组进行标准化，axis=1 表示按行标准化
        scaled_data = preprocessing.scale(reshaped_data, axis=1, with_mean=True, with_std=True, copy=True)
        # 将标准化后的二维数组重新转回三维数组
        data = scaled_data.reshape(data.shape)
    else:
        reshaped_trial_data = trial_data.reshape(-1, trial_data.shape[-1])
        scaled_trial_data = preprocessing.scale(reshaped_trial_data, axis=1, with_mean=True, with_std=True, copy=True)
        data = scaled_trial_data.reshape(data.shape)
    # convert 128 vector ---> 4*9*9 cube
    # data(40*120*128)
    for vector in data:
        temp_data = np.empty((0, 4, 8, 9))
        for i in range(0, 120):
            vector_3D = np.empty((0, 8, 9))  # 初始化一个空的二维数组，用于存放当前 vector 的处理结果
            for band in range(0, 4):
                data_2D_temp = data_1Dto2D(vector[i][band * sub_vector_len:(band + 1) * sub_vector_len])
                # print("data_2D_temp:", data_2D_temp)
                # print("data_2D_temp shape:", data_2D_temp.shape) # data_2D_temp shape: (8, 9)
                data_2D_temp = data_2D_temp.reshape(1, 8, 9)
                vector_3D = np.vstack([vector_3D, data_2D_temp])
            vector_3D = vector_3D.reshape(1, 4, 8, 9)
            temp_data = np.vstack([temp_data, vector_3D])
        temp_data = temp_data.reshape(1, 120, 4, 8, 9)
        data_3D = np.vstack([data_3D, temp_data])
    # print("final data shape:", data_3D.shape)
    # print("final data:", data_3D)
    return data_3D, arousal_labels, valence_labels

In [12]:
# 处理de
dataset_dir = "D:/huangzhiying/cross-video-emotion-recognition/cross-video-emotion-recognition/dataset/DEAP/deap_1d_result/psd"
use_baseline = "yes"
if use_baseline == "yes":
    result_dir = "D:/huangzhiying/cross-video-emotion-recognition/cross-video-emotion-recognition/dataset/DEAP/data_2d/with_base_0.5/"
    if os.path.isdir(result_dir) == False:
        os.makedirs(result_dir)
else:
    result_dir = "D:/huangzhiying/cross-video-emotion-recognition/cross-video-emotion-recognition/dataset/DEAP/data_2d/without_base_0.5/psd/"
    if os.path.isdir(result_dir) == False:
        os.makedirs(result_dir)

final_data = np.empty((0, 40, 4, 8, 9))
final_arousal_labels = np.empty((0, 40))
final_valence_labels = np.empty((0, 40))
for file in os.listdir(dataset_dir):
    print("processing: ", file, "......")
    file_path = os.path.join(dataset_dir, file)
    data, arousal_labels, valence_labels = pre_process(file_path, use_baseline)
    data = data.transpose([1, 0, 2, 3, 4])
    final_data = np.vstack([final_data, data])
    # print("1 person shape:", data.shape)
    # print("final shape:", final_data.shape)
    arousal_labels = arousal_labels.transpose([1, 0])
    valence_labels = valence_labels.transpose([1, 0])
    final_arousal_labels = np.vstack([final_arousal_labels, arousal_labels])
    final_valence_labels = np.vstack([final_valence_labels, valence_labels])
    # print("arousal shape:", arousal_labels.shape)
    # print("valence shape:", valence_labels.shape)
    # break
final_data = final_data.transpose([1, 0, 2, 3, 4])
final_arousal_labels = final_arousal_labels.transpose([1, 0]) # 在这里是一个153600的一维数组,0和1分别代表label
final_valence_labels = final_valence_labels.transpose([1, 0])

print("final shape:", final_data.shape) # (40, 120, 4, 8, 9)
for video in range(0, 40):
        print("PSD_video", str(video + 1).zfill(2), "is saving ......")
        sio.savemat(result_dir + "PSD_video" + str(video + 1).zfill(2) + ".mat",
                {"data": final_data[video], "valence_labels": final_valence_labels[video], "arousal_labels": final_arousal_labels[video]})

processing:  PSD_s01.mat ......
processing:  PSD_s02.mat ......
processing:  PSD_s03.mat ......
processing:  PSD_s04.mat ......
processing:  PSD_s05.mat ......
processing:  PSD_s06.mat ......
processing:  PSD_s07.mat ......
processing:  PSD_s08.mat ......
processing:  PSD_s09.mat ......
processing:  PSD_s10.mat ......
processing:  PSD_s11.mat ......
processing:  PSD_s12.mat ......
processing:  PSD_s13.mat ......
processing:  PSD_s14.mat ......
processing:  PSD_s15.mat ......
processing:  PSD_s16.mat ......
processing:  PSD_s17.mat ......
processing:  PSD_s18.mat ......
processing:  PSD_s19.mat ......
processing:  PSD_s20.mat ......
processing:  PSD_s21.mat ......
processing:  PSD_s22.mat ......
processing:  PSD_s23.mat ......
processing:  PSD_s24.mat ......
processing:  PSD_s25.mat ......
processing:  PSD_s26.mat ......
processing:  PSD_s27.mat ......
processing:  PSD_s28.mat ......
processing:  PSD_s29.mat ......
processing:  PSD_s30.mat ......
processing:  PSD_s31.mat ......
processi